# Reproducibility - resume

1. Create and commit two versions of dataframe -> v1, v2
2. Run flow 1 with v1 data -> crashes b/c flow 2 hasn't run
3. Run flow 2 with v2 data
4. Resume flow 1 successfully

In [26]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.WARNING)

from doltpy.core import Dolt
from doltpy.core.write import import_df
import pandas as pd

dolt = Dolt.init("foo")

df_v1 = pd.DataFrame({"A": [1,1,1], "B": [1,1,1]})
df_v2 = pd.DataFrame({"A": [1,1,1,2,2,2], "B": [1,1,1,2,2,2]})

import_df(dolt, "bar", df_v1.reset_index(), ["index"], "create")
dolt.add("bar")
dolt.commit("Initialize bar")

v1 = list(dolt.log(number="1").keys())[0]

import_df(dolt, "bar", df_v2.reset_index(), ["index"], "update")
dolt.add("bar")
dolt.commit("Add rows to bar")

v2 = list(dolt.log(number="1").keys())[0]

In [24]:
!cat succeeds_second.py

import datetime
import pickle

from metaflow import FlowSpec, step, DoltDT, Parameter, Flow
import pandas as pd
from sklearn import tree

class SucceedsSecondDemo(FlowSpec):

    bar_version = Parameter('bar-version',  help="Specifc the tag for the input version", required=True)

    @step
    def start(self):
        with DoltDT(run=self, database='foo', branch="master") as dolt:
            self.df = dolt.read_table('bar')

        first_run = Flow("SucceedsFirstDemo").latest_successful_run
        first_run_ts = datetime.datetime.strptime(first_run.finished_at, "%Y-%m-%dT%H:%M:%SZ")
        if first_run_ts < (datetime.datetime.now() - datetime.timedelta(minutes=1)):
            raise Exception("Run `FirstDemo` within one minute of `SecondDemo`")

        self.next(self.middle)

    @step
    def middle(self):
        with DoltDT(run=self, database='foo', branch="master") as dolt:
            df = self.df
            df["B"] = df["B"].map(lambda x: x*2)

            dolt.write_table(

In [28]:
!cat succeeds_first.py

import pickle

from metaflow import FlowSpec, step, DoltDT, Parameter
import pandas as pd
from sklearn import tree

class SucceedsFirstDemo(FlowSpec):

    bar_version = Parameter('bar-version',  help="Specifc the tag for the input version", required=True)

    @step
    def start(self):
        with DoltDT(run=self, database='foo', branch="master") as dolt:
            self.df = dolt.read_table('bar', commit=self.bar_version)

        self.next(self.middle)

    @step
    def middle(self):
        with DoltDT(run=self, database='foo', branch="master") as dolt:

            df = self.df
            df["B"] = df["B"].map(lambda x: x*2)

            dolt.write_table(table_name='baz', df=df, pks=['index'])

        self.next(self.end)

    @step
    def end(self):
        pass


if __name__ == '__main__':
    SucceedsFirstDemo()


In [45]:
!poetry run python3 succeeds_second.py run --bar-version $v1

Metaflow 2.2.5.post23+git20868e9 executing SucceedsSecondDemo for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-17 18:13:27.616 Workflow starting (run-id 1610936007609681):
2021-01-17 18:13:27.624 [1610936007609681/start/1 (pid 30483)] Task is starting.
2021-01-17 18:13:28.998 [1610936007609681/start/1 (pid 30483)] 01-17 18:13:28 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 18:13:29.046 [1610936007609681/start/1 (pid 30483)] 01-17 18:13:29 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 18:13:29.219 [1610936007609681/start/1 (pid 30483)] 01-17 18:13:29 doltpy.core.dolt INFO     * master                                        	bp7f940q1914r898fcntcnhhaihg8c7t
2021-01-17 18:13:29.397 [1610936007609681/start/1 (pid 30483)] 
2021-01-17 18:13:29.398 [1610936007609681/start/1 (pid 30483)] 01-17 18:13:29 

In [46]:
!poetry run python3 succeeds_first.py run --bar-version $v2

Metaflow 2.2.5.post23+git20868e9 executing SucceedsFirstDemo for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-17 18:13:45.921 Workflow starting (run-id 1610936025914102):
2021-01-17 18:13:45.929 [1610936025914102/start/1 (pid 30526)] Task is starting.
2021-01-17 18:13:47.168 [1610936025914102/start/1 (pid 30526)] 01-17 18:13:47 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 18:13:47.196 [1610936025914102/start/1 (pid 30526)] 01-17 18:13:47 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 18:13:47.259 [1610936025914102/start/1 (pid 30526)] 01-17 18:13:47 doltpy.core.dolt INFO     * master                                        	bp7f940q1914r898fcntcnhhaihg8c7t
2021-01-17 18:13:47.541 [1610936025914102/start/1 (pid 30526)] 
2021-01-17 18:13:47.541 [1610936025914102/start/1 (pid 30526)] 01-17 18:13:47 d

2021-01-17 18:13:50.690 [1610936025914102/middle/2 (pid 30560)] Task finished successfully.
2021-01-17 18:13:50.699 [1610936025914102/end/3 (pid 30600)] Task is starting.
2021-01-17 18:13:51.891 [1610936025914102/end/3 (pid 30600)] 01-17 18:13:51 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
2021-01-17 18:13:51.898 [1610936025914102/end/3 (pid 30600)] 01-17 18:13:51 doltpy.core.system_helpers INFO     No processes to clean up, exiting
2021-01-17 18:13:52.056 [1610936025914102/end/3 (pid 30600)] Task finished successfully.
2021-01-17 18:13:52.056 Done!
01-17 18:13:52 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
01-17 18:13:52 doltpy.core.system_helpers INFO     No processes to clean up, exiting


In [47]:
!poetry run python3 succeeds_second.py resume start

Metaflow 2.2.5.post23+git20868e9 executing SucceedsSecondDemo for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-17 18:14:07.621 Gathering required information to resume run (this may take a bit of time)...
2021-01-17 18:14:07.631 Workflow starting (run-id 1610936047620343):
2021-01-17 18:14:07.639 [1610936047620343/start/1 (pid 30615)] Task is starting.
2021-01-17 18:14:08.711 [1610936047620343/start/1 (pid 30615)] 01-17 18:14:08 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 18:14:08.737 [1610936047620343/start/1 (pid 30615)] 01-17 18:14:08 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 18:14:08.804 [1610936047620343/start/1 (pid 30615)] 01-17 18:14:08 doltpy.core.dolt INFO     * master                                        	aif4600d4clfvsv2da5630bp4m4nrthk
2021-01-17 18:14:08.981 [161093604762034

2021-01-17 18:14:11.801 [1610936047620343/middle/2 (pid 30649)] Task finished successfully.
2021-01-17 18:14:11.808 [1610936047620343/end/3 (pid 30689)] Task is starting.
2021-01-17 18:14:12.995 [1610936047620343/end/3 (pid 30689)] 01-17 18:14:12 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
2021-01-17 18:14:13.003 [1610936047620343/end/3 (pid 30689)] 01-17 18:14:13 doltpy.core.system_helpers INFO     No processes to clean up, exiting
2021-01-17 18:14:13.166 [1610936047620343/end/3 (pid 30689)] Task finished successfully.
2021-01-17 18:14:13.166 Done!
01-17 18:14:13 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
01-17 18:14:13 doltpy.core.system_helpers INFO     No processes to clean up, exiting
